Association Rule Mining (Apriori Algorithm)

<span style = "font-size:larger">Q1. (a) Generate frequent itemsets from the given dataset.</span>

In [ ]:
import pandas as pd
import numpy as np

xl = pd.ExcelFile("bread.xlsx")
df = xl.parse('Sheet1')
df[['Date', 'Time','Transaction','Item']] = df['Date,Time,Transaction,Item'].str.split(',', expand=True)
df = df.drop(['Date,Time,Transaction,Item'],axis = 1)
df = df[df['Item']!='NONE']
df

In [4]:
min_sup = 0.04
min_conf = 0.5

transactions = set(df['Transaction'])
total_transac = len(transactions)

#variable to store all the frequenct itemsets
fr_itemsets = {}

In [5]:
#function to return the number of transactions having all the items in a given list

def validTransactions(itemset):
    common_transac = set(df['Transaction'])
    for x in itemset:
        item_transac = set(df[df['Item'] == x]['Transaction'])
        common_transac = common_transac.intersection(item_transac)
    return len(common_transac)

In [6]:
#Inserting frequent one itemsets
items = df['Item'].unique()

fr1 = []
for i in items:
    count = validTransactions([i])
    sup = count/total_transac
    if sup>=min_sup:
        fr1.append([i])

fr_itemsets[1] = fr1

In [ ]:
#Generating all frequent itemsets using two previous frequent itemsets F(k) = F(k-1)*F(k-1)

k = 2

while len(fr_itemsets[k-1])>1:
    n = len(fr_itemsets[k-1])
    fr_k = []
    for i in range(n-1):
        for j in range(i+1,n):
            mergeable = True
            for z in range(k-2):
                if(fr_itemsets[k-1][i][z]!=fr_itemsets[k-1][j][z]):
                    mergeable = False
                    break

            if(mergeable):
                new_itemset = fr_itemsets[k-1][i][:k-2]
                new_itemset.append(fr_itemsets[k-1][i][-1])
                new_itemset.append(fr_itemsets[k-1][j][-1])
                new_itemset.sort()
                
                sup = validTransactions(new_itemset)/total_transac
                
                if(sup>=min_sup):
                    fr_k.append(new_itemset)
                    
    fr_itemsets[k] = fr_k
    k+=1      

#as the the last value of k has not candidates
k-=1

for x in fr_itemsets:
    if(len(fr_itemsets[x])>0):
        print("The Frequent "+str(x)+"-Itemsets are :")
        print(fr_itemsets[x])
        print("\n")

<span style = "font-size:larger">Q1. (b) Derive association rules from the given dataset.</span>

In [8]:
def generateSubsets(val):
    if val == []:
        return [[]]
    x = generateSubsets(val[1:])
    return x + [[val[0]] + y for y in x]

In [ ]:
#Generating all association rules and pruning based on confidence

rules = []
for ind in range(1,k):
    itemsets = fr_itemsets[ind]
    for i in itemsets:
        subsets = generateSubsets(i)
        for j in subsets:
            if len(j)>0 and len(j)<len(i):
                left = j
                
                #complement of left
                right = list(x for x in i if x not in left)
                
                conf = validTransactions(left+right)/validTransactions(left)
                if(conf >= min_conf):
                    rules.append([left,right,conf])

print("The valid Assocation Rules are :\n")
for i in rules:
    print(str(i[0]) + " --> "+str(i[1])+ "   (Confidence = "+str(i[2])+")")
                